In [1]:
##importing necessary packages 
import pandas as pd
import numpy as np
import scipy.optimize as opt
import statsmodels.api as sm
import scipy.stats as stats
from geopy.distance import geodesic
from geopy import distance

In [2]:
#reading in the Stata File
radio_merger = pd.read_csv('radio_merger_data.csv')
radio_merger.head(n=6)

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,price,hhi_target,num_stations_buyer,population_target,corp_owner_buyer
0,2007,1,1,46.592512,-92.549564,44.375073,-92.039543,157763.91,80,3,21676,0
1,2007,2,2,32.578185,-85.349003,33.025375,-86.059702,1472463.20,376,1,11539,0
2,2007,3,3,30.639867,-88.254450,31.122499,-87.766408,3786333.90,129,1,182265,0
3,2007,4,4,38.956806,-94.683236,36.196946,-94.006823,473291.74,188,20,203065,0
4,2007,5,5,41.054082,-73.536216,40.909898,-73.457023,1840579.00,284,0,1493350,0
5,2007,6,6,39.223416,-106.882770,39.269849,-107.023350,1104347.40,51,1,17148,0


In [3]:
radio_merger.describe()

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,price,hhi_target,num_stations_buyer,population_target,corp_owner_buyer
count,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,9.900000e+01,99.000000,99.000000,9.900000e+01,99.000000
mean,2007.545455,25.454545,25.454545,36.909475,-94.392918,37.476019,-94.365015,4.138339e+06,96.353535,18.828283,3.229588e+05,0.020202
std,0.500464,14.709620,14.709620,5.068201,13.957259,4.853933,13.695796,1.468443e+07,82.242463,69.837719,1.098850e+06,0.141407
min,2007.000000,1.000000,1.000000,25.558428,-122.710600,25.369355,-122.433420,5.724072e+04,10.000000,0.000000,2.553000e+03,0.000000
25%,2007.000000,13.000000,13.000000,32.886576,-104.497680,33.687825,-104.876615,4.913584e+05,54.500000,1.000000,2.169800e+04,0.000000
50%,2008.000000,25.000000,25.000000,36.653256,-92.662675,36.900901,-92.122592,1.013110e+06,72.000000,2.000000,4.994800e+04,0.000000
75%,2008.000000,37.500000,37.500000,40.858790,-83.813483,41.145355,-84.212299,2.330153e+06,91.000000,5.500000,1.538550e+05,0.000000
max,2008.000000,54.000000,54.000000,48.704839,-71.411500,48.906401,-70.018443,1.390000e+08,388.000000,591.000000,9.818605e+06,1.000000


In [4]:
##We scale price and population as suggested in the problem: 
radio_merger['ln_price'] = np.log((radio_merger['price']) / 1000)
radio_merger['ln_popu'] = np.log((radio_merger['population_target']) / 1000)
radio_merger = radio_merger.drop(columns = ['population_target', 'price']) ##not needed
radio_merger

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,hhi_target,num_stations_buyer,corp_owner_buyer,ln_price,ln_popu
0,2007,1,1,46.592512,-92.549564,44.375073,-92.039543,80,3,0,5.061100,3.076206
1,2007,2,2,32.578185,-85.349003,33.025375,-86.059702,376,1,0,7.294692,2.445733
2,2007,3,3,30.639867,-88.254450,31.122499,-87.766408,129,1,0,8.239154,5.205462
3,2007,4,4,38.956806,-94.683236,36.196946,-94.006823,188,20,0,6.159712,5.313526
4,2007,5,5,41.054082,-73.536216,40.909898,-73.457023,284,0,0,7.517835,7.308777
...,...,...,...,...,...,...,...,...,...,...,...,...
94,2008,50,50,38.824690,-104.562030,38.270870,-85.786112,98,0,0,8.019393,6.608130
95,2008,51,51,39.609596,-79.967623,38.820973,-80.042884,34,16,0,6.969570,3.381165
96,2008,52,52,35.294405,-118.905170,37.104785,-107.919850,164,4,0,5.534486,3.938353
97,2008,53,53,40.278498,-76.875207,40.107720,-77.782639,80,2,0,6.787249,5.008085


In [5]:
##adding distance (km)
radio_merger['distance'] = radio_merger.apply(lambda row: (distance.geodesic((row.buyer_lat, row.buyer_long), (row.target_lat, row.target_long)).km), axis = 1)
radio_merger

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,hhi_target,num_stations_buyer,corp_owner_buyer,ln_price,ln_popu,distance
0,2007,1,1,46.592512,-92.549564,44.375073,-92.039543,80,3,0,5.061100,3.076206,249.652058
1,2007,2,2,32.578185,-85.349003,33.025375,-86.059702,376,1,0,7.294692,2.445733,83.008467
2,2007,3,3,30.639867,-88.254450,31.122499,-87.766408,129,1,0,8.239154,5.205462,70.999830
3,2007,4,4,38.956806,-94.683236,36.196946,-94.006823,188,20,0,6.159712,5.313526,312.082957
4,2007,5,5,41.054082,-73.536216,40.909898,-73.457023,284,0,0,7.517835,7.308777,17.343800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2008,50,50,38.824690,-104.562030,38.270870,-85.786112,98,0,0,8.019393,6.608130,1635.068840
95,2008,51,51,39.609596,-79.967623,38.820973,-80.042884,34,16,0,6.969570,3.381165,87.793532
96,2008,52,52,35.294405,-118.905170,37.104785,-107.919850,164,4,0,5.534486,3.938353,1007.567555
97,2008,53,53,40.278498,-76.875207,40.107720,-77.782639,80,2,0,6.787249,5.008085,79.562803


In [6]:
radio_merger.year.describe() ##how many years? we have two years, 2007 and 2008. so we create sub-data for them

count      99.000000
mean     2007.545455
std         0.500464
min      2007.000000
25%      2007.000000
50%      2008.000000
75%      2008.000000
max      2008.000000
Name: year, dtype: float64

In [7]:
#separating the yearly markets
rd_2007 = radio_merger[radio_merger['year'] == 2007] ##market for 2007
rd_2008 = radio_merger[radio_merger['year'] == 2008] ##market for 2008
years = [rd_2007, rd_2008]

In [8]:
##now separate buyer and target characeristics, and create list comprehension for counterfactual loops 
buyer_charact = ['year', 'buyer_lat', 'buyer_long', 'buyer_id', 'corp_owner_buyer', 'num_stations_buyer']
target_charact = ['target_lat', 'target_long', 'target_id', 'ln_price', 'hhi_target', 'ln_popu']
countfct_rad = pd.DataFrame()
countfct_rad = [cf[buyer_charact].iloc[b].values.tolist() + cf[target_charact].iloc[t].values.tolist() \
                for cf in years for b in range(len(cf) - 1) for t in range(b + 1, len(cf))]
countfct_rad = pd.DataFrame(countfct_rad, columns = buyer_charact + target_charact)
countfct_rad ##2421 rows

,year,buyer_lat,buyer_long,buyer_id,corp_owner_buyer,num_stations_buyer,target_lat,target_long,target_id,ln_price,hhi_target,ln_popu
0,2007.0,46.592512,-92.549564,1.0,0.0,3.0,33.025375,-86.059702,2.0,7.294692,376.0,2.445733
1,2007.0,46.592512,-92.549564,1.0,0.0,3.0,31.122499,-87.766408,3.0,8.239154,129.0,5.205462
2,2007.0,46.592512,-92.549564,1.0,0.0,3.0,36.196946,-94.006823,4.0,6.159712,188.0,5.313526
3,2007.0,46.592512,-92.549564,1.0,0.0,3.0,40.909898,-73.457023,5.0,7.517835,284.0,7.308777
4,2007.0,46.592512,-92.549564,1.0,0.0,3.0,39.269849,-107.023350,6.0,7.007010,51.0,2.841882
...,...,...,...,...,...,...,...,...,...,...,...,...
2416,2008.0,39.609596,-79.967623,51.0,0.0,16.0,40.107720,-77.782639,53.0,6.787249,80.0,5.008085
2417,2008.0,39.609596,-79.967623,51.0,0.0,16.0,31.745025,-87.138810,54.0,5.534486,70.0,3.138446
2418,2008.0,35.294405,-118.905170,52.0,0.0,4.0,40.107720,-77.782639,53.0,6.787249,80.0,5.008085
2419,2008.0,35.294405,-118.905170,52.0,0.0,4.0,31.745025,-87.138810,54.0,5.534486,70.0,3.138446


In [9]:
##we can see that there is no distance. so we add distance (km) column
countfct_rad['distance'] = countfct_rad.apply(lambda row: (distance.geodesic((row.buyer_lat, row.buyer_long), \
                                                                (row.target_lat, row.target_long)).km), axis = 1)
countfct_rad #now we have distance

,year,buyer_lat,buyer_long,buyer_id,corp_owner_buyer,num_stations_buyer,target_lat,target_long,target_id,ln_price,hhi_target,ln_popu,distance
0,2007.0,46.592512,-92.549564,1.0,0.0,3.0,33.025375,-86.059702,2.0,7.294692,376.0,2.445733,1604.192191
1,2007.0,46.592512,-92.549564,1.0,0.0,3.0,31.122499,-87.766408,3.0,8.239154,129.0,5.205462,1765.973939
2,2007.0,46.592512,-92.549564,1.0,0.0,3.0,36.196946,-94.006823,4.0,6.159712,188.0,5.313526,1160.907565
3,2007.0,46.592512,-92.549564,1.0,0.0,3.0,40.909898,-73.457023,5.0,7.517835,284.0,7.308777,1656.940898
4,2007.0,46.592512,-92.549564,1.0,0.0,3.0,39.269849,-107.023350,6.0,7.007010,51.0,2.841882,1430.892416
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2416,2008.0,39.609596,-79.967623,51.0,0.0,16.0,40.107720,-77.782639,53.0,6.787249,80.0,5.008085,194.970963
2417,2008.0,39.609596,-79.967623,51.0,0.0,16.0,31.745025,-87.138810,54.0,5.534486,70.0,3.138446,1086.749994
2418,2008.0,35.294405,-118.905170,52.0,0.0,4.0,40.107720,-77.782639,53.0,6.787249,80.0,5.008085,3632.981877
2419,2008.0,35.294405,-118.905170,52.0,0.0,4.0,31.745025,-87.138810,54.0,5.534486,70.0,3.138446,2964.530070


I will define the payoff functions for the two models, mergers with and without tranders 

In [10]:
##payoff function without transfers
def first_payoff(m, i, params):
    '''
   This function helps to calculate the payoffs of the mergers
    Args:
        m: the merger dataset
        i: we use it for iteration purposes 
        params: parameters
    Returns:
        function: payoff
    '''
    alpha = params[0]
    beta = params[1]

    function = m['num_stations_buyer'].iloc[i] * m['ln_popu'].iloc[i] + alpha * m['corp_owner_buyer'].iloc[i] * m['ln_popu'].iloc[i] + beta * m['distance'].iloc[i]

    return(function)

In [11]:
##payoff function with transfers
def second_payoff(z, i, lemmas):
    '''
    This is the function we use to calculate the payoffs of the mergers with transfers.
    Args:
        z: mergers data
        i: for purpuses of row iterations
        lemmas: parameters
    Returns:
        funct: payoff
    '''
    beta = lemmas[3]
    gamma = lemmas[2]
    alpha = lemmas[1]
    delta = lemmas[0]

    funct = delta * z['num_stations_buyer'].iloc[i] * z['ln_popu'].iloc[i] + alpha * z['corp_owner_buyer'].iloc[i] * z['ln_popu'].iloc[i] + gamma * z['hhi_target']+ beta * z['distance'].iloc[i]

    return(funct)

Now I calculate the payoffs

In [12]:
# estimating actual payoffs for model 1
Params = (0.1, -0.5)
for i in range(len(rd_2007)):
    payoff_actual_2007 = [first_payoff(m = rd_2007, i = i, params = Params)] ##for 2007

for i in range(len(rd_2008)):
    payoff_actual_2008 = [first_payoff(m = rd_2008, i = i, params = Params)] ##for 2008

In [13]:
## now estimating counterfactuals payoffs for model 1
for i in range(0, (len(rd_2007) * (len(rd_2007) - 1))):
    payoff_cf_2007 = [first_payoff(m = countfct_rad, i = i, params = Params)]

for i in range((len(rd_2007) * (len(rd_2007) - 1)), len(countfct_rad)):
    payoff_cf_2008 = [first_payoff(m = countfct_rad, i = i, params = Params)]

In [14]:
# estimating actual payoffs for model 2
lemmas_2 = (0.4, 0.5, 0.5, -1)
# 2007 actual
for i in range(len(rd_2007)):
    pactual_2007_2 = [second_payoff(z = rd_2007, i = i, lemmas = lemmas_2)] ##for 2007

for n in range(len(rd_2008)):
    pactual_2008_2 = [second_payoff(z = rd_2008, i = i, lemmas = lemmas_2)] ##for 2008

In [15]:
# estimating counterfactuals payoffs for model 2
for i in range(0, (len(rd_2007) * (len(rd_2007) - 1))):
    pcf_2007_2 = [second_payoff(z = countfct_rad, i = i, lemmas = lemmas_2)]

for i in range((len(rd_2007) * (len(rd_2007) - 1)), len(countfct_rad)):
    pcf_2008_2 = [second_payoff(z = countfct_rad, i = i, lemmas = lemmas_2)]

I now define the objective functions to be maximised and print the results

In [16]:
## objective fucntion for model 1

def obj_funct(payoff_actual_2007, payoff_actual_2008, payoff_cf_2007, payoff_cf_2008, Params):
    '''
    This is the function used to calculate the obj function to be maximised
    Args:
        m:
    Returns:
        max_score: maximum score
    '''
    max_score = 0

    for i in [[payoff_actual_2007, payoff_cf_2007], [payoff_actual_2008, payoff_cf_2008]]:
        for j in range(len(i[0])):
            for k in range(len(i[0])):
                if i[0][j] + i[0][k] >= i[1][k: j] + i[1][k: (j - 1)]:
                    max_score = max_score - 1

    return(max_score)
model_1 = opt.minimize(obj_funct, Params, args = (payoff_actual_2007, payoff_actual_2008, payoff_cf_2007, payoff_cf_2008), method = 'Nelder-Mead', options = {'maxiter': 5000})
print("Results for the model 1:", model_1)

Results for the model 1:  final_simplex: (array([[ 0.1       , -0.5       ],
       [ 0.10001953, -0.5       ],
       [ 0.1       , -0.50009766]]), array([-1., -1., -1.]))
           fun: -1.0
       message: 'Optimization terminated successfully.'
          nfev: 35
           nit: 9
        status: 0
       success: True
             x: array([ 0.1, -0.5])


<ipython-input-16-fa26c657253f>:16: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if i[0][j] + i[0][k] >= i[1][k: j] + i[1][k: (j - 1)]:


In [19]:
##objective function for model 2
def obj_funct_2(pactual_2007_2, pactual_2008_2, pcf_2007_2, pcf_2008_2, lemmas_2):
    '''
    This is the function used to calculate the 2nd obj function to be maximised
    Args:
        z:
    Returns:
        max_score: max score
    '''
    max_score = 0
    
    for i in [[pactual_2007_2, pcf_2007_2], [pactual_2008_2, pcf_2008_2]]:
        for j in range(len(i[0])):
            for k in range(len(i[0])):
                if (i[0][j] - i[1][k: j] >= radio_merger.ln_price[j] - countfct_rad.ln_price[k]) & (i[0][k] - i[1][k: (j - 1)] >= radio_merger.ln_price[k] - countfct_rad.ln_price[j]):
                    max_score = max_score - 1

    return(max_score)

model_2 = opt.minimize(obj_funct_2, lemmas_2, args = (pactual_2007_2, pactual_2008_2, pcf_2007_2, pcf_2008_2), method = 'Nelder-Mead', options = {'maxiter': 5000})
print(model_2)

<ipython-input-19-5fc71116f3ac>:15: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (i[0][j] - i[1][k: j] >= radio_merger.ln_price[j] - countfct_rad.ln_price[k]) & (i[0][k] - i[1][k: (j - 1)] >= radio_merger.ln_price[k] - countfct_rad.ln_price[j]):


ValueError: operands could not be broadcast together with shapes (1,54) (0,) 

I TRIED RECTIFYING THIS ERROR, ALL TO NO AVAIL!!